Set Animal IDs and Experiment IDs

In [14]:
id_dict = {
    # 'LT177': ['20220517_130039']} # ,
    'LT178': ['20220517_134340']} # ,
    # 'LT179': ['20220527_142920'],
    # 'LT180': ['20220527_151659']
    # }

Get User Input for Visual Area Boundaries on Sign Maps

In [15]:
import numpy as np
from skimage.morphology import skeletonize
import cv2
import matplotlib.pyplot as plt
from tifffile import imread
    

def get_signmap_boundaries(signmap, dilation_factor):
    '''
    Get binary image of sign map boundaries by:
        1) Thresholding sign map around zero to detect boundary regions.
        2) Skeletonizing thresholded sign map to reduce thickness of certain boundary regions.
        3) Dilating skeletonized image by dilation_factor to make boundary thickness uniform and suitable for display purposes.
    '''
    # Detect outer edges of sign map (floats bordering nans) and set to 0
    signmap_threshold = signmap.copy()
    kernel_edge = np.array([[-1, -1, -1],
                            [-1, 1, -1],
                            [-1, -1 , -1]])
    for row in np.arange(1, signmap_threshold.shape[0] - 1):
        for col in np.arange(1, signmap_threshold.shape[1] - 1):
            curr_idx = np.ix_((row - 1, row, row + 1),(col - 1, col, col + 1))
            if not (np.isnan(signmap[row,col]) and
                        np.isnan(np.tensordot(kernel_edge, signmap_threshold[curr_idx], 
                                              axes=((0,1),(0,1))))):
                signmap_threshold[curr_idx] = 0

    # Threshold sign map around 0 to detect boundaries
    np.add.at(signmap_threshold, np.logical_or(signmap < -0.35, signmap > 0.35), np.nan)

    # Skeletonize, then dilate thresholded sign map
    signmap_skeleton = skeletonize(~np.isnan(signmap_threshold))
    kernel_dilate = np.ones((dilation_factor, dilation_factor), np.uint8)
    signmap_boundaries = cv2.dilate(signmap_skeleton.astype('float'), kernel_dilate, iterations=1)
    signmap_boundaries[signmap_boundaries == 0] = np.nan
    return signmap_boundaries


# Set visual areas to include in analysis
visual_areas = ['V1', 'LM', 'Li', 'AL', 'RL', 'AM', 'PM']

data_path = '//Users//ryan.gorzek//Documents//Trachtenberg Lab//Data//Liming//{ms}//{exp}//'

for mouse in id_dict:
    for experiment in id_dict[mouse]:
        exp_path = data_path.format(ms=mouse, exp=experiment)
        ref_frame = imread(exp_path + 'refframe.tif').squeeze()
        signmap = np.loadtxt(exp_path + 'signmap.csv')
        # Get sign map boundaries, use dilation factor of 3 (good boundary thickness)
        signmap_boundaries = get_signmap_boundaries(signmap, 3)
        # for vis_ar in visual_areas:
        %matplotlib qt
        fig, ax = plt.subplots()
        ax.imshow(ref_frame, cmap='gray')
        ax.imshow(signmap, cmap='RdBu_r', clim=[-1,1], alpha=0.3)
        # ax.imshow(signmap_boundaries, cmap='Greys', clim=[0,1], alpha=1)
        ax.axis('off')
        fig.set_size_inches(10, 10)

In [4]:
for key in id_dict:
    print(key)

LT177
LT178
LT179
LT180
